# `swift_too` module

## Swift_ObsQuery example - querying past *Swift* observations

### API version = 1.2,`swifttools` version = 3.0.8

#### Author: Jamie A. Kennea (Penn State)

The Swift_ObsQuery class allows for querying the database of observations have have already been performed by Swift, otherwise known as the "As-Flown Science Timeline" (AFST). Note this will only fetch observations that have already been performed, not scheduled observations. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from swifttools.swift_too import ObsQuery

### Constructing the query


Note: As of `swifttools` version 2.2, you do not need to pass `username` or `shared_secret` keywords, they will default to anonymous. This is the recommended usage for anything except a `TOO` request.

First example, how often has Swift observed the binary system SS 433? Well, I can't remember the RA/Dec off the top of my head, so let's look it up...

#### New features in `swifttools` 2.3

`swifttools` 2.3 supports a new class called `Swift_Resolve`. You can use this to do name resolution, i.e. converting the name of a target into coordinates. However, it's also built into classes that take RA/Dec now, so you can just pass it using the `name` parameters.

Another new feature of 2.3, there are now shorthand names for classes, so you can omit the `Swift_` when calling the class, changing it to just `ObsQuery`.

In [ ]:
query = ObsQuery()
query.name = "SS 433"

Now you can see the coordinates, as either RA/Dec or `astropy`'s `SkyCoord`, using the attributes `ra`, `dec` and `skycoord` (if `astropy` is installed). For example:

In [ ]:
query.skycoord

RA/Dec is stored by the TOO API in decimal degrees, in J2000 epoch as this is the epoch Swift uses

In [ ]:
print(f"RA/Dec(J2000) = {query.ra:.4f}, {query.dec:.4f}")

Looks legit. However, we can also set RA/Dec the old fashioned way

In [ ]:
query.ra, query.dec = 287.9565, 4.9827

Note that skycoord will remain correct even if you changed the ra and dec properties.

In [ ]:
query.skycoord


Swift_ObsQuery has a default search radius which is....

In [ ]:
print(f"Default search radius = {query.radius:.3f} degrees")

That's 12 arcminutes, which is the approximate field of view of Swift's X-ray Telescope (XRT). We can narrow that down a bit, so we only get matches that are in the center of the field of view (FOV), and also in UVOT which has a smaller FOV.

In [ ]:
query.radius = 5 / 60  # 5 arc-minutes, as the units for radius are degrees

#### New in `swifttools 3.0.8` `astropy` units support

In `3.0.8` you can now define radius using units, so let's try that again this time using astropy.

In [ ]:
import astropy.units as u

query.radius = 5 * u.arcmin

print(f"Search radius = {query.radius:.3f} degrees")

Note that the API still stores the value internally in degrees. 

### Submitting the query 

OK let's query the Swift timeline to see how many observations it's taken. This might take a few seconds to process. `query` method will return `True` if everything is OK, `False` if there is a problem. If there is a problem, simply look at the contents of the `status` attribute.

In [ ]:
if query.submit():
    print("Success!")
else:
    print(f"Fail or timeout? {query.status}")

Looks like that worked, let's take a look at the `status` attribute anyway.

In [ ]:
query.status

### Let's do that again, but this time, in a more compact way

The above was extremely verbose, we can compress it all down to a single line by passing parameters as arguments:

In [ ]:
query = ObsQuery(name="SS433", radius=5 * u.arcmin)

### Examining the results of the query

So how many observations has Swift taken of this target?

In [ ]:
print(f"This many: {len(query)}")

That's a lot of damage. Here's a thing to remember, every entry in this is a single snapshot of observation. As Swift is in a low Earth orbit, it means that a snapshot is typically max 30 mins, or sometimes a bit longer (~44 mins), so a long exposure will consist of multiple snapshots. Observations are grouped by obsid (a 12 digit number with leading zeros), so snapshots with the same obsid are part of the same planned observation.

In [ ]:
query

Wow that is a lot of observations. 

### Pointing Accuracy

Here's an interesting thing about Swift, it doesn't point very accurately. This is because the ACS system sacrifices accuracy for speed. The goal is to get the object of interest into the field of view of XRT and UVOT, not at the boresight. As a result, the pointing direction be typically up to 3 arcminutes off the requested pointing direction. Note that for each entry listed above, we give an ra and dec value, so let's check out the variation:

In [ ]:
plt.figure()
plt.plot([float(entry.ra) for entry in query], [float(entry.dec) for entry in query], "+")
plt.plot([entry.ra_object for entry in query], [entry.dec_object for entry in query], "X")
plt.xlabel("RA(J2000)")
plt.ylabel("Dec(J2000)")
_ = plt.title(f"{query.name} pointing scatter")

As you can see there's a lot of variation of the pointing direction. Each entry also has a values `ra_object`, `dec_object`, which give the decimal degrees values of the requested pointing direction for each observation. This typically will be the coordinates of the Target of the Observation, but sometimes if offsets are applied for any reason, it might differ.

The `ra_object`/`dec_object` values aren't necessarily going to be for the object you queried on. In fact there can be multiple values of ra_point/dec_point if the queried field lies inside multiple pointings.

Although the RA/Dec are returned in J2000 decimal degrees, they're also returned as skycoords. However, `ra_object` and `dec_object` are not so we will have to convert those to SkyCoords manually.

In [ ]:
query[0].skycoord

Sometimes `ra_object`, `dec_object` cannot be determined, so the value will be 'None'. I'm going to filter those out so we can do some comparing.

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

sc = SkyCoord([entry.skycoord for entry in query if entry.ra_point is not None])
scp = SkyCoord(
    [entry.ra_point for entry in query if entry.ra_point is not None],
    [entry.dec_point for entry in query if entry.ra_point is not None],
    frame="fk5",
    unit=(u.deg, u.deg),
)

I made an array of ra_point/dec_point so we can evaluate how accurately Swift actually pointed at this target. Let's make a histogram of the pointing offsets.

In [ ]:
plt.figure()
plt.hist(sc.separation(scp).arcmin, bins=30)
plt.ylabel("Number of pointings")
plt.xlabel("Offset from requested pointing direction (arc-minutes)")
print(f"Median offset value = {np.median(sc.separation(scp).arcmin):0.2f} arc-minutes")

So you'll see that the median offset here is around 2 arc-minutes, with a pretty big scatter, and there may even be some large outliers. 


### Grouping Snapshots into Observations 

Let's take a look at an individual entry to see what information is being returned by this query.

In [ ]:
query[0]

So some useful information here. Firstly remember each entry represents a snapshot of Swift data, that is data taken in a single orbit of observations. Typically data that you obtain from the SDC will be grouped by observation, and those observations can contain many snapshots. Observations have a unique target ID (target ID) and segment (seg) numbers. These typically are combined into a Observation Number (obsnum), which in the SDC format will look like a concatibation of the target ID, and segment, with padding zeros, e.g.:

In [ ]:
print(f"Target ID = {query[0].targetid}, segment = {query[0].seg}, ObservationID = {query[0].obsnum}")

If you're interested in all the observations under a particular Observation ID, then there's a property called "observations" that contains a dictionary of all observations on an Observation ID basis. Let's look at the summary of this dictionary by just printing out all the entries.

In [ ]:
query.observations

You can see that now the summary shows the details for an entire observation, with the begin and end times being those of associated with the first and last observation of that Observation ID, and the exposure time being the total. Importantly due to orbit gaps, the exposure time is not just end minus begin. Each entry in the observations dictionary contains details on the individual segments also. Note that Observation ID is a string, given as it is formatted with padding zeros. For example:

In [ ]:
query.observations["00035190015"]

If we want to see what snapshots make up a particular observation, it's easy:

In [ ]:
query.observations["00035190015"].snapshots

You can query the exposure and other infomation for the combined snapshots in the Observation ID.

In [ ]:
query.observations["00035190015"].exposure

Note that the result here is a datetime timedelta object. You can easily get the seconds as an integer or convert to an astropy TimeDelta object.

In [ ]:
query.observations["00035190015"].exposure.seconds

In [ ]:
from astropy.time import TimeDelta

TimeDelta(query.observations["00035190015"].exposure)

Note that there is no RA/Dec (ra/dec) for an observation, only the requested pointing direction (ra_point/dec_point), because actual RA/Dec will be different for each snapshot, so delve into the individual snapshots for those. 

In [ ]:
query.observations["00035190015"].ra_point, query.observations["00035190015"].dec_point

# Instrument Configuration 

All observations for a given Observation ID will have the same instrument configuration. Let's check those out.

In [ ]:
print(f"XRT mode = {query.observations['00035190015'].xrt}, UVOT mode = {query.observations['00035190015'].uvot}")

In this case the XRT mode is `Auto`, which means that XRT itself decides whether to be in PC or WT mode, based on the brightness of sources in the central 200x200 pixels of the detector, roughly the central 8.5 arcmin x 8.5 arcmin box. Because of this we can't determine what mode XRT will have actually taken the data in without looking at it. However for many observation, the mode is fixed, here you will see results like so:

In [ ]:
print(f"XRT mode = {query.observations['00035190037'].xrt}")

As this is PC mode, we can guarantee that the data are taken in PC mode.

For UVOT the mode above is a hex number `0x20ed`. There are a large number of modes that can be used with UVOT, given it's many different combinations of filters, exposure windows, etc. Luckily you can query what this mode means using the `UVOTMode` class. We can quickly display a table showing details of the mode as follows:

In [ ]:
from swifttools.swift_too import UVOTMode

UVOTMode(uvotmode=query.observations["00035190015"].uvot)

Finally: It's also important to remember that a specific UVOT mode does not guarantee data was taken with the expected filters, so always check the data.

# Querying by Time
If instead of querying by position or target, you are interested in querying the *Swift* AFST by time of observation, for instance if you want to know where *Swift* was pointed at a given time, this can also be easily done (most of the time). You just define pass `begin` for your given time, without specifying `end`. `begin` should either be in `datetime` format, e.g. 

In [ ]:
from datetime import datetime

# define your time of interest
timestamp = datetime(2021, 4, 19, 13, 35, 27)

or you can just specify the timestamp in ISO format as a string, e.g.

In [ ]:
timestamp = "2021-04-19 13:35:27"

You can even use an `astropy` `Time` class if you like, which is handy if you want to use time formats other than regular UT. Note that `swift_too` does not depend on `astropy`, so it returns values in datetime format for UT.

In [ ]:
from astropy.time import Time

timestamp = Time(59323.56629, format="mjd")

...noting that timestamps are in UT timezone. Note that whatever format you pass the date as, it will get converted internally to a UT `datetime.datetime`.

In [ ]:
query = ObsQuery(begin=timestamp)

In [ ]:
if query.status.status == "Accepted":
    print("All Good!")
else:
    print(f"Not good: {query.status}")

Ok, what did we get?

In [ ]:
query

A single observation, that was performed during our time of interest, as expected. Show me the deets!

In [ ]:
print(f"Swift pointing RA, Dec, Roll: {query[0].ra, query[0].dec, query[0].roll}")

But wait! This is the *settled* pointing, if your time of interest happens to be during the slew to this object, then these pointing coordinates are *not* valid.

In [ ]:
if timestamp > query[0].settle:
    print("Good to go!")
else:
    print(
        f"Swift was slewing during your time of interest. \nTo get pointing information for this time, \
consult the attitude file associated with obsid {query[0].obsnum}."
    )

*Swift* is slewing ~15% of the time, so most of the time this method will quickly get you the answer you want.

You can of course query over a range of times, simply by specifying the `end` parameter, so:

In [ ]:
ObsQuery(begin=timestamp, end=Time(59323.56629 + 0.1, format="mjd"))

As an alternative, you can also request the length of time to return. This by default is the number of days, so `length=1` will return 1 days of results, however you can also pass a `datetime.timedelta` object or astropy `TimeDelta` if you like. Here's how we fetch the first hour of observations on MJD 59000.

In [ ]:
ObsQuery(begin=Time(59000, format="mjd"), length=TimeDelta(3600, format="sec"))

# Querying by Target ID and Observation Number

Target ID is the number assigned to a specific target, so if you have that number and you want to see how often a target with that target ID has been observation, you can specify it in the search. Note that some targets have mulitple target IDs assigned to them, so we always recommend doing an RA/Dec search if you want to know about all the observations that have been taken.

In [ ]:
query = ObsQuery(targetid=35000)

In [ ]:
query

You can also query by Observation Number. This number comes in two formats, the SDC format, which is always a 11 character long string consisting of the target ID (8 characters) and segment (3 characters) with leading zeros. So for a target ID 35000 and segment 7, the SDC format is `00035000007`. The other format is the one used by Swift itself, in which the observation number is a 32-bit word, in which the target ID is the lower three bytes, and the segment number is the highest significance byte. Here's a demonstration of how this works:

In [ ]:
targetid = 35000
segment = 7
obsnum = f"{targetid:08}{segment:03}"
obsnumsc = targetid + (segment << 24)
print(f"SDC format: {obsnum} Swift format: {obsnumsc}")

Either can be used, but if the SDC format is desired, it must be passed as a string.

In [ ]:
ObsQuery(obsnum="00035000007")

In [ ]:
ObsQuery(obsnum=117475512)

Note that the TOO API internally stores as `targetid` and `segment`, but transparanently converts it for you.

### A note about time formats

#### New features in `swifttools` 2.4

All times returned by `ObsQuery` come out in a time system that is derived from Swift's internal clock. The problem with this is that this clock, although close to UTC, is not actually UTC, due to the lack of handling of leap seconds, and also a slow drift in the clock itself. However, you can now correct this using the `clock_correct()` method. Let's do that:

In [ ]:
query = ObsQuery(targetid=12345)
query

In [ ]:
query.clock_correct()
query

So the table above now shows times that are specifically labelled as UTC. You'll note that the times now have fractions of a second, which is due to the clock correction that is applied. You can see the value of this clock correction (which includes both leap seconds and clock drift corrections), by looking at one of the date values, e.g.:

In [ ]:
query[0].begin

So for this observation, the UT Correction Factor (UTCF) is -24.012318s. For more detail on this correction, please take a look at the `Swift_Clock` and the notebook explaining that.